In [1]:
import csv
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import IncrementalPCA
from tabulate import tabulate
import tensorflow as tf
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import AllChem
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


In [2]:
#initial loading

ddi_fp = "drugbank\drugbank.tab"
ddi = pd.read_csv(ddi_fp, sep='\t')

kaggle_fp = "SMILES-Kaggle\chembl_22_clean_1576904_sorted_std_final.smi"
smiles = pd.read_csv(kaggle_fp, sep='\t')

drug_names_fp = "drugs.txt"
drug_names = pd.read_csv(drug_names_fp, sep='\t')

ddi["Y"] = ddi["Y"].astype("category")
ddi["Map"] = ddi["Map"].astype("category")

interaction_counts = pd.DataFrame(ddi['Y'].value_counts().rename_axis('value').reset_index(name='count')).sort_values(by='count', ascending=False)
interaction_counts['row_num'] = interaction_counts.index + 1
interaction_counts['log_count'] = np.log(interaction_counts['count'])

interaction_types = ddi[['Y','Map']].drop_duplicates(subset=['Y'])

ddi = ddi.drop("Map",axis=1)

ddi.head(10)

,ID1,ID2,Y,X1,X2
0,DB04571,DB00460,1,CC1=CC2=CC3=C(OC(=O)C=C3C)C(C)=C2O1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
1,DB00855,DB00460,1,NCC(=O)CCC(O)=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
2,DB09536,DB00460,1,O=[Ti]=O,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
3,DB01600,DB00460,1,CC(C(O)=O)C1=CC=C(S1)C(=O)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
4,DB09000,DB00460,1,CC(CN(C)C)CN1C2=CC=CC=C2SC2=C1C=C(C=C2)C#N,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
5,DB11630,DB00460,1,OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
6,DB00553,DB00460,1,COC1=C2OC(=O)C=CC2=CC2=C1OC=C2,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
7,DB06261,DB00460,1,[H]N([H])CC(=O)CCC(=O)OCCCCCC,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
8,DB01878,DB00460,1,O=C(C1=CC=CC=C1)C1=CC=CC=C1,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...
9,DB00140,DB00460,1,CC1=C(C)C=C2N(C[C@H](O)[C@H](O)[C@H](O)CO)C3=N...,COC(=O)CCC1=C2NC(\C=C3/N=C(/C=C4\N\C(=C/C5=N/C...


In [3]:
ddi.shape

(191808, 5)

In [4]:
def delist(list_of_lists):
    list_of_strings = []
    for inner_list in list_of_lists:
        string = inner_list[0]
        list_of_strings.append(string)
    return list_of_strings

In [5]:
#preprocessing

old = pd.DataFrame()
old["total"] = ddi['ID1'].value_counts()
old = old.reset_index()
old.columns = ['ID', 'count'] 
new = pd.DataFrame()
new["total"] = ddi['ID2'].value_counts()
new = new.reset_index()
new.columns = ['ID', 'count'] 

drug_counts = pd.merge(old,new,how='outer',on='ID').fillna(0)
drug_counts['total'] = drug_counts['count_x'] + drug_counts['count_y']

drug_counts = drug_counts.sort_values(by='total')
drug_counts_one = pd.DataFrame(drug_counts[drug_counts['total']==1]['ID'])

ddi_proc = ddi[ ~ddi['ID1'].isin(drug_counts_one['ID'])]
ddi_proc = ddi_proc[ ~ddi_proc['ID2'].isin(drug_counts_one['ID'])]
ddi_proc = ddi_proc[ddi_proc['X1']!="OC1=CC=CC(=C1)C-1=C2\CCC(=N2)\C(=C2/N\C(\C=C2)=C(/C2=N/C(/C=C2)=C(\C2=CC=C\-1N2)C1=CC(O)=CC=C1)C1=CC(O)=CC=C1)\C1=CC(O)=CC=C1"]


In [8]:
#create main datasets

data = dc.data.NumpyDataset(X=ddi_proc[['X1','X2']], y=ddi[['Y']])
df = data.to_dataframe()
df = df.sample(frac=1).reset_index(drop=True)

X_one = delist(df[["X1"]].values.tolist())
X_two = delist(df[["X2"]].values.tolist())


search_string = "nan"

count = X_one.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_one.")
count = X_two.count(search_string)
if count > 0:
    print(f"'{search_string}' found {count} times in X_two.")

In [10]:
#featurize using circular fingerprint

cf_featurizer = dc.feat.CircularFingerprint()

#other way to featurize a molecule
#cm_featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=False)

#Need Discovery access to run these
#df.rename(columns={'ids':'col'},  inplace=True)
#df_x1_cf = pd.DataFrame(cf_featurizer(X_one))
#df_x1_cf.rename(columns=lambda x: "x1_cf_"+str(x+1), inplace=True)
#df_x1_cf['col'] = df_x1_cf.index
#df_x2_cf = pd.DataFrame(cf_featurizer(X_two))
#df_x2_cf.rename(columns=lambda x: "x2_cf_"+str(x+1), inplace=True)
#df_x2_cf['col'] = df_x2_cf.index

#df = df.merge(df_x1_cf, on="col")
#df = df.merge(df_x2_cf, on="col")

#example process with just 1000 rows
df_example = df.head(1000)
df_example = df_example.reset_index()
df_example.rename(columns={'index':'col'},  inplace=True)

df_example_x1_cf = pd.DataFrame(cf_featurizer(X_one[0:1000]))
df_example_x1_cf.rename(columns=lambda x: "x1_cf_"+str(x+1), inplace=True)
df_example_x1_cf['col'] = df_example_x1_cf.index
df_example_x2_cf = pd.DataFrame(cf_featurizer(X_two[0:1000]))
df_example_x2_cf.rename(columns=lambda x: "x2_cf_"+str(x+1), inplace=True)
df_example_x2_cf['col'] = df_example_x2_cf.index

df_example = df_example.merge(df_example_x1_cf, on="col")
df_example = df_example.merge(df_example_x2_cf, on="col")

df_example.shape

[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerator
[15:25:35] DEPRECATION WARNING: please use MorganGenerat

(1000, 4102)

In [12]:
#Basic dimensionality reduction down to 20 components from 4097

df_X = df_example.iloc[:,5:]
df_Y = to_categorical(df_example["y"])

n_components = 20
batch_size = 20

ipca = IncrementalPCA(n_components=n_components)

for i in range(0, df_X.shape[0], batch_size):
    X_batch = df_X[i:i + batch_size]
    ipca.partial_fit(X_batch)

X_transformed = ipca.transform(df_X)

#prove that X's shape has changed
#print("Original shape:", df_X.shape)
#print("Transformed shape:", X_transformed.shape)

df_X_proc = pd.DataFrame(X_transformed)

X_train, X_test, y_train, y_test = train_test_split(df_X_proc, df_Y, test_size=0.2, random_state=1337)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [14]:
#basic/rough neural network implementation

model = keras.models.Sequential([
    Dense(20, activation='relu'),
    keras.layers.Dropout(0.2),
    Dense(40, activation='relu'),
    Dense(80, activation='relu'),
    Dense(160, activation='relu'),
    Dense(86, activation='softmax')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=20, validation_split=0.1)

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy}")

Epoch 1/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0511 - loss: 0.5405 - val_accuracy: 0.3000 - val_loss: 0.0517
Epoch 2/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3590 - loss: 0.0515 - val_accuracy: 0.3000 - val_loss: 0.0441
Epoch 3/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3406 - loss: 0.0439 - val_accuracy: 0.3000 - val_loss: 0.0424
Epoch 4/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3413 - loss: 0.0451 - val_accuracy: 0.3500 - val_loss: 0.0406
Epoch 5/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3817 - loss: 0.0421 - val_accuracy: 0.4250 - val_loss: 0.0400
Epoch 6/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3700 - loss: 0.0397 - val_accuracy: 0.3875 - val_loss: 0.0386
Epoch 7/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3810 - loss: 0.0394 - val_accuracy: 0.3875 - val_loss: 0.0379
Epoch 8/20
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4208 - loss: 0.0389 - val_accuracy: 0.4750 - val_loss